In [9]:
#wine quality sample

def train(in_alpha, in_lr_ratio):
    import logging
    import warnings
    
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split
    
    import mlflow
    import mlflow.sklearn
    from mlflow.models import infer_signature
    
    logging.basicConfig(level = logging.WARN)
    logger = logging.getLogger(__name__)
    
    
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    #read the data
    
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "unable to downlod the data. Error: %s", e
        )
        
    #split the data
    train, test = train_test_split(data, test_size=0.20, random_state=42)
    
    #prediction column is quantity 
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    #set default values if no alpha and l1_ratio provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)
        
    if float(in_lr_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_lr_ratio)
        
    #mlflow
    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        #evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        #print out metrics
        print(f"ElasticNet Model (alpha={alpha:f}, l1_ratio = {l1_ratio:f}):")
        print(" RMSE: %s" % rmse)
        print(" MAE : %s" % mae)
        print(" R2: %s" % r2)
        
        #infer model signature
        predictions = lr.predict(train_x)
        signature = infer_signature(train_x, predictions)
        
        #log parameter, metrics and model with mlflow
        
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
        mlflow.sklearn.log_model(lr, "model", signature=signature)
    
    
    

In [10]:
train(0.5,0.5)

ElasticNet Model (alpha=0.500000, l1_ratio = 0.500000):
 RMSE: 0.762794521686023
 MAE : 0.6188630472018415
 R2: 0.10963999179642603


In [11]:
train(0.2,0.8)

ElasticNet Model (alpha=0.200000, l1_ratio = 0.800000):
 RMSE: 0.7265642494268115
 MAE : 0.5845090917094489
 R2: 0.1922098310640783


In [12]:
train(0.2,0.1)

ElasticNet Model (alpha=0.200000, l1_ratio = 0.100000):
 RMSE: 0.6929185588573523
 MAE : 0.5543829927884041
 R2: 0.2652917761622785
